<a href="https://colab.research.google.com/github/rayxtn/amazon_review_classification/blob/main/amazon_review_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk import corpus, tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer , PorterStemmer , porter
import re
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sen_analayzer = SentimentIntensityAnalyzer()

In [6]:
test_text =  "this is a good movie !"
sen_analayzer.polarity_scores(test_text)

{'neg': 0.0, 'neu': 0.484, 'pos': 0.516, 'compound': 0.4926}

In [4]:
test_text2 = "this is too bad ! "
sen_analayzer.polarity_scores(test_text2)

{'neg': 0.558, 'neu': 0.442, 'pos': 0.0, 'compound': -0.5848}